## データの前処理

In [1]:
import numpy as np
import pandas as pd
np.random.seed(443)
from datetime import datetime as dt

In [2]:
msm2014 = pd.read_csv("MSM/RJAA_2014.csv")
msm2015 = pd.read_csv("MSM/RJAA_2015.csv")
msm2016 = pd.read_csv("MSM/RJAA_2016.csv")
msm2017 = pd.read_csv("MSM/RJAA_2017.csv")
metar2014 = pd.read_csv("METAR/RJAA_2014_edit.csv")
metar2015 = pd.read_csv("METAR/RJAA_2015_edit.csv")
metar2016 = pd.read_csv("METAR/RJAA_2016_edit.csv")
metar2017 = pd.read_csv("METAR/RJAA_2017_edit.csv")

In [3]:
# 複数年の結合
MSM = pd.concat([msm2014,msm2015,msm2016,msm2017]).reset_index(drop=True)
METAR = pd.concat([metar2014,metar2015,metar2016,metar2017]).reset_index(drop=True)
METAR.WX.fillna("NSW", inplace=True)

In [4]:
# 欠損値を確認
chk = MSM.apply(pd.isnull).apply(sum, axis=0)
chk[ chk[:] != 0 ]

Series([], dtype: int64)

In [5]:
# 欠損値を確認
chk = METAR.apply(pd.isnull).apply(sum, axis=0)
chk[ chk[:] != 0 ]

Series([], dtype: int64)

### MSM気象要素の生成
相対湿度から露点温度、および湿数を計算

In [6]:
## 気温と湿度から露点温度を計算
def Dewpoint_Temperature(t, rh):
    # エラー処理
    rh *= 1.0
    rh[ rh < 0.1 ] = 0.1
    # 露点温度を計算
    return t - np.log(rh/100) / (0.00047 * t - 0.073)

## 気温と湿度から湿数を計算
def Dewpoint_Depression(t, rh):
    td = Dewpoint_Temperature(t, rh)
    return T_minus_Td(t,td)

## 湿数 = 気温・露点温度差
def T_minus_Td(t, td):
    return t - td

In [7]:
## 露点温度、湿数
MSM["TD_SFC"] = Dewpoint_Temperature( MSM.TMP_SFC, MSM.RH_SFC )
MSM["TmTD_SFC"] = T_minus_Td( MSM.TMP_SFC, MSM.TD_SFC )
#MSM["TmTD_1000"] = Dewpoint_Depression( MSM.TMP_1000, MSM.RH_1000 )

地上風は風向・風速にする

In [8]:
## UV成分から風向風速を計算
def UV_to_SpdDir(u,v):
    wspd = np.sqrt( u**2 + v**2 )
    wdir = np.rad2deg( np.arctan2(u,v) )+180
    
    # 風向が0になった場合、360にする
    # 風速が0になった場合、風向は0にしておく
    wdir[ wdir==0 ] = 360.0
    wdir[ wspd==0 ] = 0.0
    return wspd, wdir

## 360度方位を8方位に変換
def Deg_to_Dir8(deg):
    # 風向0はVRBにする
    deg[ deg==0 ] = 720
    dirname = ['N','NE','E','SE','S','SW','W','NW','VRB']
    
    deg = deg + 22.5
    deg[ deg>360 ] -= 360
    deg = (deg/45).astype(np.int64)
    dir8 = deg.apply( lambda x: dirname[x] )
    return dir8

## 360度方位を16方位に変換
def Deg_to_Dir16(deg):
    # 風向0はVRBにする
    deg[ deg==0 ] = 720
    dirname = ['N','NNE','NE','ENE','E','ESE','SE','SSE','S','SSW','SW','WSW','W','WNW','NW','NNW','VRB']
    
    deg = deg + 11.25
    deg[ deg>360 ] -= 360
    deg = (deg/22.5).astype(np.int64)
    dir16 = deg.apply( lambda x: dirname[x] )
    return dir16

In [9]:
## 風向・風速
wspd, wdir = UV_to_SpdDir( MSM.UGRD_SFC, MSM.VGRD_SFC )
MSM["WSPD_SFC"] = wspd
#MSM["WDIR_SFC"] = wdir
MSM["WDIR_SFC"] = Deg_to_Dir8(wdir)
#MSM["WDIR_SFC"] = Deg_to_Dir16(wdir)

In [10]:
## 下層の気圧面の風速
MSM["WSPD_1000"] = np.sqrt( MSM.UGRD_1000**2 + MSM.VGRD_1000**2 )
MSM["WSPD_975"] = np.sqrt( MSM.UGRD_975**2 + MSM.VGRD_975**2 )
MSM["WSPD_950"] = np.sqrt( MSM.UGRD_950**2 + MSM.VGRD_950**2 )

霧の発生しやすい時間帯があると考えられるので、時間帯を特徴量に加える  
また霧の発生しやすい季節もあると考えられるので、季節を特徴量に加える

In [11]:
## 日時を処理する関数
def DateTime_Hour(x):
    date_time = dt.strptime(x, "%Y/%m/%d %H:%M:%S")
    return date_time.hour
def DateTime_Month(x):
    date_time = dt.strptime(x, "%Y/%m/%d %H:%M:%S")
    return date_time.month
def Time_Range(hour):
    trname = ["0-2","3-5","6-8","9-11","12-14","15-17","18-20","21-23"]
    tr = (hour/3).astype(np.int64).apply( lambda x: trname[x] )
    return tr

In [12]:
## 季節（ここでは月）と時間帯（24時間を8つに分割）
MSM["MONTH"] = MSM.DateTime.apply(DateTime_Month)
MSM["TimeRange"] = Time_Range( MSM.DateTime.apply(DateTime_Hour) )

気圧と気温と露点温度は、1時間前からの変化量を特徴量に加える

In [13]:
# 気圧と気温と露点温度の、1時間前からの変化量
MSM["D_PRES_SFC"] = MSM.PRES_SFC - MSM.PRES_SFC.shift(1)
MSM["D_TMP_SFC"] = MSM.TMP_SFC - MSM.TMP_SFC.shift(1)
MSM["D_TD_SFC"] = MSM.TD_SFC - MSM.TD_SFC.shift(1)

最下層の風の鉛直シアと気温の鉛直プロファイルを特徴量に加える

In [14]:
## 2つの気圧面の間の、風の鉛直シア
def Vertical_Wind_Shear(u1, v1, h1, u2, v2, h2):
    # 2気圧面の厚さ：m → ft
    depth = M_to_FT( h2 - h1 )
    
    # 風のベクトル差：m/s → kt
    d_u = MPS_to_KT( u2 - u1 )
    d_v = MPS_to_KT( v2 - v1 )
    
    # 風の鉛直シア：kt per 1000ft
    vws = np.sqrt( d_u**2 + d_v**2 ) / depth * 1000
    return vws

## m/sをktに変換
def MPS_to_KT(x):
    return x/0.514444

## mをftに変換
def M_to_FT(x):
    return x/0.3048

## 最下層の安定度　→　2つの気圧面の間の気温差（気温減率 × -1 = 気温増加量）
def Atmospheric_Stability(t1, h1, t2, h2):
    # 気温増加量：C per 100m
    return ( t2 - t1 ) / ( h2 - h1 ) * 100

In [15]:
## 最下層の風の鉛直シア (kt / 1000ft)
MSM["LL_VWS1"] = Vertical_Wind_Shear( MSM.UGRD_1000, MSM.VGRD_1000, MSM.HGT_1000, MSM.UGRD_975, MSM.VGRD_975, MSM.HGT_975 )
MSM["LL_VWS2"] = Vertical_Wind_Shear( MSM.UGRD_975, MSM.VGRD_975, MSM.HGT_975, MSM.UGRD_950, MSM.VGRD_950, MSM.HGT_950 )

# 当初は地上気圧が1000hPa未満の場合、975-950hPaの鉛直シアを計算する予定だったが、値の連続性が悪くなりそうなのでやめた
# vws1 = Vertical_Wind_Shear( MSM.UGRD_1000, MSM.VGRD_1000, MSM.HGT_1000, MSM.UGRD_975, MSM.VGRD_975, MSM.HGT_975 )
# vws2 = Vertical_Wind_Shear( MSM.UGRD_975, MSM.VGRD_975, MSM.HGT_975, MSM.UGRD_950, MSM.VGRD_950, MSM.HGT_950 )
# MSM["LL_VWS"] = vws1
# MSM["LL_VWS"][ MSM["HGT_1000"] < 0 ] = vws2[ MSM["HGT_1000"] < 0 ]

## 大気の安定度 (C / 100m)
MSM["LL_STBL1"] = Atmospheric_Stability( MSM.TMP_1000, MSM.HGT_1000, MSM.TMP_975, MSM.HGT_975 )
MSM["LL_STBL2"] = Atmospheric_Stability( MSM.TMP_975, MSM.HGT_975, MSM.TMP_950, MSM.HGT_950 )


前線霧を想定し、下層のある気圧面と最下層の気圧面の気温差と降水量の積を特徴量に加える

In [16]:
## 2つの気圧面の間の気温差と、降水量の積
#  前線面の上から相対的に暖かい雨が降る状況を想定：前線霧
#  気温増加量と降水量の積がマイナスにならないようにする
def Warmer_Rain(t1, h1, t2, h2, prcp):
    C = Atmospheric_Stability(t1, h1, t2, h2)
    C[ C<0 ] = 0.0

    ## この処理は採用しない
    # C は100mあたりの気温増加量
    # 乾燥断熱減率は1.0C per 100mなので、1.0を足せばCは必ず正の値になるはず
    # もし1.0を足しても負の場合は、エラー処理としてゼロにする
    #C = C + 1.0
    #C[ C<0 ] = 0.0
    
    return C * prcp

## This is old one.
#def Warmer_Rain(t1, t2, prcp):
#    return ( t2 - t1 ) * prcp

In [17]:
## 大気の安定度と降水量の積（前線霧のための特徴量）
MSM["WARMER_RA"] = Warmer_Rain( MSM.TMP_1000, MSM.HGT_1000, MSM.TMP_950, MSM.HGT_950, MSM.APCP_SFC )

# 評価用
# MSM["WARMER_RA1"] = Warmer_Rain( MSM.TMP_1000, MSM.HGT_1000, MSM.TMP_850, MSM.HGT_850, MSM.APCP_SFC )
# MSM["WARMER_RA2"] = Warmer_Rain( MSM.TMP_1000, MSM.HGT_1000, MSM.TMP_925, MSM.HGT_925, MSM.APCP_SFC )
# MSM["WARMER_RA3"] = Warmer_Rain( MSM.TMP_1000, MSM.HGT_1000, MSM.TMP_950, MSM.HGT_950, MSM.APCP_SFC )

### METARの整形  
1時間前の視程を特徴量として使えるようにする

In [18]:
METAR["VIS_P1HR"] = METAR.VIS.shift(1)

視程に基づいて、霧の有無を判別  
また視程の値に基づいて、カテゴリー分け

In [19]:
## 霧の有無、通常は1000m未満が霧だが任意の値を指定可能
def FG_or_not(vis, border=1000):
    fg = np.zeros( vis.shape[0] ).astype(np.int64)
    fg[ vis < border ] = 1
    return fg

## 視程の値の範囲でカテゴリー分け
# CAT 0:0-400m, 1:400-800m, 2:800-1600m, 3:1600-3200m, 4:3200-5000m, 5:5000m以上
def VIS_Category(vis):
    cat = np.full( vis.shape[0], 0 )
    cat[ vis < 5000 ] = 1
    cat[ vis < 3200 ] = 2
    cat[ vis < 1600 ] = 3
    cat[ vis < 800 ] = 4
#     cat[ vis < 400 ] = 5
    return cat

In [20]:
## 霧の閾値を1600mとして、霧の有無を判定
METAR["FG"] = FG_or_not( METAR.VIS, border=1600 )
## 視程値でカテゴリー分け
# CAT 4:0-800m, 3:800-1600m, 2:1600-3200m, 1:3200-5000m, 0:5000m以上
METAR["VIS_CAT"] = VIS_Category( METAR.VIS )

In [21]:
# 雪が降っている時のデータは使わない
METAR.drop( METAR.index[ METAR.WX.str.contains('SN') ], inplace=True )

### METARとMSMの結合

In [22]:
DATA = pd.merge(METAR, MSM, on="DateTime", how="inner")
DATA.dropna(inplace=True)
DATA.reset_index(drop=True, inplace=True)

In [23]:
# 欠損値を確認
chk = DATA.apply(pd.isnull).apply(sum, axis=0)
chk[ chk[:] != 0 ]

Series([], dtype: int64)

In [24]:
# 学習用データ(2014-2016)と予測実験用データ(2017)に分割
border = DATA.index[ DATA.DateTime.str.startswith('2017') ][0]
DATA_TRAIN = DATA.iloc[:border]
DATA_TEST = DATA.iloc[border:].reset_index(drop=True)

In [25]:
# ファイルに書き出す
DATA_TRAIN.to_csv("train_data.csv", index=False)
DATA_TEST.to_csv("test_data.csv", index=False)
# DATA.to_csv("all_data.csv", index=False)